In [4]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re

In [5]:
# location of raw data files
input_path = "~/datasets/raw/"

In [6]:
#read LABEVENTS.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'INPUTEVENTS_MV.csv',dtype = "object")

In [7]:
#read D_ITEMS.csv file
df1 = pd.read_csv(filepath_or_buffer=input_path+'D_ITEMS.csv',dtype = "object")

In [8]:
# map the labels for ITEMD 
df=df.merge(df1[['ITEMID','LABEL']], how='left', left_on='ITEMID', right_on='ITEMID')

In [9]:
# Print datatype and non-null data points for each column
df.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3618991 entries, 0 to 3618990
Data columns (total 32 columns):
ROW_ID                           3618991 non-null object
SUBJECT_ID                       3618991 non-null object
HADM_ID                          3618991 non-null object
ICUSTAY_ID                       3617827 non-null object
STARTTIME                        3618991 non-null object
ENDTIME                          3618991 non-null object
ITEMID                           3618991 non-null object
AMOUNT                           3618991 non-null object
AMOUNTUOM                        3618991 non-null object
RATE                             2042852 non-null object
RATEUOM                          2042852 non-null object
STORETIME                        3618991 non-null object
CGID                             3618991 non-null object
ORDERID                          3618991 non-null object
LINKORDERID                      3618991 non-null object
ORDERCATEGORYNAME               

In [10]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL
0,241,27063,139787,223259,2133-02-05 06:29:00,2133-02-05 08:45:00,225166,6.774531824,mEq,NaN,...,0,0,1,Rewritten,NaN,RN,2133-02-05 12:52:00,10,.050000001,Potassium Chloride
1,242,27063,139787,223259,2133-02-05 05:34:00,2133-02-05 06:30:00,225944,28.1329972,ml,30.142497,...,0,0,0,FinishedRunning,NaN,NaN,NaN,28.132998,30.255817,Sterile Water
2,243,27063,139787,223259,2133-02-05 05:34:00,2133-02-05 06:30:00,225166,2.813299944,mEq,NaN,...,0,0,0,FinishedRunning,NaN,NaN,NaN,2.8132997,.050426364,Potassium Chloride
3,244,27063,139787,223259,2133-02-03 12:00:00,2133-02-03 12:01:00,225893,1,dose,NaN,...,0,0,2,Rewritten,RN,NaN,2133-02-03 17:06:00,1,1,Piperacillin/Tazobactam (Zosyn)
4,245,27063,139787,223259,2133-02-03 12:00:00,2133-02-03 12:01:00,220949,100,ml,NaN,...,0,0,2,Rewritten,RN,NaN,2133-02-03 17:06:00,100,0,Dextrose 5%


In [11]:
#Dropping Row_ID 
df = df.drop(labels='ROW_ID',axis = 'columns')

In [12]:
#Dropping redundant columns
df = df.drop(labels=['STARTTIME','ENDTIME','COMMENTS_EDITEDBY','COMMENTS_CANCELEDBY','COMMENTS_DATE','CGID','ORDERID','LINKORDERID','ISOPENBAG','CONTINUEINNEXTDEPT','CANCELREASON','ORIGINALAMOUNT','ORIGINALRATE'],axis = 'columns')

In [13]:
# Print datatype and non-null data points for each column
df.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3618991 entries, 0 to 3618990
Data columns (total 18 columns):
SUBJECT_ID                       3618991 non-null object
HADM_ID                          3618991 non-null object
ICUSTAY_ID                       3617827 non-null object
ITEMID                           3618991 non-null object
AMOUNT                           3618991 non-null object
AMOUNTUOM                        3618991 non-null object
RATE                             2042852 non-null object
RATEUOM                          2042852 non-null object
STORETIME                        3618991 non-null object
ORDERCATEGORYNAME                3618991 non-null object
SECONDARYORDERCATEGORYNAME       2635563 non-null object
ORDERCOMPONENTTYPEDESCRIPTION    3618991 non-null object
ORDERCATEGORYDESCRIPTION         3618991 non-null object
PATIENTWEIGHT                    3618991 non-null object
TOTALAMOUNT                      3084964 non-null object
TOTALAMOUNTUOM                  

In [14]:
df = df.fillna('NaN')

In [15]:
# merge ITEMID, VALUE, UOM into single token
df['AMOUNT_VALUE_UOM'] = df['ITEMID']+'-'+df['AMOUNT']+'-'+df['AMOUNTUOM']
df['RATE_VALUE_UOM'] = df['ITEMID']+'-'+df['RATE']+'-'+df['RATEUOM']
df['FLUID_VALUE_UOM'] = 'FLUID'+'-'+df['TOTALAMOUNT']+'-'+df['TOTALAMOUNTUOM']
df['PATIENTWEIGHT'] = 'WEIGHT'+'-'+df['PATIENTWEIGHT']

In [16]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,STORETIME,ORDERCATEGORYNAME,...,ORDERCOMPONENTTYPEDESCRIPTION,ORDERCATEGORYDESCRIPTION,PATIENTWEIGHT,TOTALAMOUNT,TOTALAMOUNTUOM,STATUSDESCRIPTION,LABEL,AMOUNT_VALUE_UOM,RATE_VALUE_UOM,FLUID_VALUE_UOM
0,27063,139787,223259,225166,6.774531824,mEq,NaN,NaN,2133-02-05 06:27:00,02-Fluids (Crystalloids),...,Additives ...,Continuous IV,WEIGHT-83.2,100,ml,Rewritten,Potassium Chloride,225166-6.774531824-mEq,225166-NaN-NaN,FLUID-100-ml
1,27063,139787,223259,225944,28.1329972,ml,30.142497,mL/hour,2133-02-05 12:57:00,02-Fluids (Crystalloids),...,Main order parameter,Continuous IV,WEIGHT-83.2,100,ml,FinishedRunning,Sterile Water,225944-28.1329972-ml,225944-30.142497-mL/hour,FLUID-100-ml
2,27063,139787,223259,225166,2.813299944,mEq,NaN,NaN,2133-02-05 12:57:00,02-Fluids (Crystalloids),...,Additives ...,Continuous IV,WEIGHT-83.2,100,ml,FinishedRunning,Potassium Chloride,225166-2.813299944-mEq,225166-NaN-NaN,FLUID-100-ml
3,27063,139787,223259,225893,1,dose,NaN,NaN,2133-02-03 13:21:00,08-Antibiotics (IV),...,Main order parameter,Drug Push,WEIGHT-83.2,100,ml,Rewritten,Piperacillin/Tazobactam (Zosyn),225893-1-dose,225893-NaN-NaN,FLUID-100-ml
4,27063,139787,223259,220949,100,ml,NaN,NaN,2133-02-03 13:21:00,08-Antibiotics (IV),...,Mixed solution,Drug Push,WEIGHT-83.2,100,ml,Rewritten,Dextrose 5%,220949-100-ml,220949-NaN-NaN,FLUID-100-ml


In [17]:
# if value if missing then replace merge tokens with 'NaN'
df.loc[df['AMOUNT']=='NaN','AMOUNT_VALUE_UOM'] = 'NaN'
df.loc[df['RATE']=='NaN','RATE_VALUE_UOM'] = 'NaN'
df.loc[df['TOTALAMOUNT']=='NaN','FLUID_VALUE_UOM'] = 'NaN'

In [18]:
# Drop redundant columns
df = df.drop(labels=['ITEMID','AMOUNT','AMOUNTUOM','RATE','RATEUOM','TOTALAMOUNT','TOTALAMOUNTUOM','PATIENTWEIGHT'],axis = 'columns')

In [19]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STORETIME,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCOMPONENTTYPEDESCRIPTION,ORDERCATEGORYDESCRIPTION,STATUSDESCRIPTION,LABEL,AMOUNT_VALUE_UOM,RATE_VALUE_UOM,FLUID_VALUE_UOM
0,27063,139787,223259,2133-02-05 06:27:00,02-Fluids (Crystalloids),Additive (Crystalloid),Additives ...,Continuous IV,Rewritten,Potassium Chloride,225166-6.774531824-mEq,NaN,FLUID-100-ml
1,27063,139787,223259,2133-02-05 12:57:00,02-Fluids (Crystalloids),Additive (Crystalloid),Main order parameter,Continuous IV,FinishedRunning,Sterile Water,225944-28.1329972-ml,225944-30.142497-mL/hour,FLUID-100-ml
2,27063,139787,223259,2133-02-05 12:57:00,02-Fluids (Crystalloids),Additive (Crystalloid),Additives ...,Continuous IV,FinishedRunning,Potassium Chloride,225166-2.813299944-mEq,NaN,FLUID-100-ml
3,27063,139787,223259,2133-02-03 13:21:00,08-Antibiotics (IV),02-Fluids (Crystalloids),Main order parameter,Drug Push,Rewritten,Piperacillin/Tazobactam (Zosyn),225893-1-dose,NaN,FLUID-100-ml
4,27063,139787,223259,2133-02-03 13:21:00,08-Antibiotics (IV),02-Fluids (Crystalloids),Mixed solution,Drug Push,Rewritten,Dextrose 5%,220949-100-ml,NaN,FLUID-100-ml


In [20]:
# Removce any speacial charecters from the free text columns
df['ORDERCATEGORYNAME'] = df['ORDERCATEGORYNAME'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x)) 

df['SECONDARYORDERCATEGORYNAME'] = df['SECONDARYORDERCATEGORYNAME'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x)) 

df['ORDERCOMPONENTTYPEDESCRIPTION'] = df['ORDERCOMPONENTTYPEDESCRIPTION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['ORDERCATEGORYDESCRIPTION'] = df['ORDERCATEGORYDESCRIPTION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['STATUSDESCRIPTION'] = df['STATUSDESCRIPTION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

In [21]:
# create inputevents_mv by mergeing all the columns on whitespace
df['inputevents_mv'] = df['LABEL']+' '+df['ORDERCATEGORYNAME']+' '+df['SECONDARYORDERCATEGORYNAME']+' '+df['ORDERCOMPONENTTYPEDESCRIPTION']+' '+df['ORDERCATEGORYDESCRIPTION']+' '+df['STATUSDESCRIPTION']+' '+df['AMOUNT_VALUE_UOM']+' '+df['RATE_VALUE_UOM']+' '+df['FLUID_VALUE_UOM']

In [22]:
# drop redundant columns
df = df.drop(labels=['LABEL','ORDERCATEGORYNAME','SECONDARYORDERCATEGORYNAME','ORDERCOMPONENTTYPEDESCRIPTION','ORDERCATEGORYDESCRIPTION','STATUSDESCRIPTION','AMOUNT_VALUE_UOM','RATE_VALUE_UOM','FLUID_VALUE_UOM'], axis = 'columns')

In [23]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STORETIME,inputevents_mv
0,27063,139787,223259,2133-02-05 06:27:00,Potassium Chloride 02-Fluids Crystalloids Addi...
1,27063,139787,223259,2133-02-05 12:57:00,Sterile Water 02-Fluids Crystalloids Additive ...
2,27063,139787,223259,2133-02-05 12:57:00,Potassium Chloride 02-Fluids Crystalloids Addi...
3,27063,139787,223259,2133-02-03 13:21:00,Piperacillin/Tazobactam (Zosyn) 08-Antibiotics...
4,27063,139787,223259,2133-02-03 13:21:00,Dextrose 5% 08-Antibiotics IV 02-Fluids Crysta...


In [24]:
# split on white spaceand convert tp list
df['inputevents_mv'] = df['inputevents_mv'].map(lambda x: x.split())

In [25]:
#Remove 'NaN' from the inputevents_mv
df['inputevents_mv'] = df['inputevents_mv'].map(lambda x: list(filter(lambda a: a!='NaN',x)))

In [26]:
df.head(100)

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STORETIME,inputevents_mv
0,27063,139787,223259,2133-02-05 06:27:00,"[Potassium, Chloride, 02-Fluids, Crystalloids,..."
1,27063,139787,223259,2133-02-05 12:57:00,"[Sterile, Water, 02-Fluids, Crystalloids, Addi..."
2,27063,139787,223259,2133-02-05 12:57:00,"[Potassium, Chloride, 02-Fluids, Crystalloids,..."
3,27063,139787,223259,2133-02-03 13:21:00,"[Piperacillin/Tazobactam, (Zosyn), 08-Antibiot..."
4,27063,139787,223259,2133-02-03 13:21:00,"[Dextrose, 5%, 08-Antibiotics, IV, 02-Fluids, ..."
5,27063,139787,223259,2133-02-05 06:27:00,"[Sterile, Water, 02-Fluids, Crystalloids, Addi..."
6,27063,139787,223259,2133-02-05 06:27:00,"[Potassium, Chloride, 02-Fluids, Crystalloids,..."
7,27063,139787,223259,2133-02-05 12:58:00,"[Sterile, Water, 02-Fluids, Crystalloids, Addi..."
8,27063,139787,223259,2133-02-05 12:58:00,"[Potassium, Chloride, 02-Fluids, Crystalloids,..."
9,27063,139787,223259,2133-02-05 06:27:00,"[Sterile, Water, 02-Fluids, Crystalloids, Addi..."


In [27]:
# create output path
mypath_output = "../../datasets/inputevents_mv/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [28]:
# export the dataframe to JSON format
df.to_json(mypath_output+"inputevents_mv.json",orient = 'records') 